                                                One Example

https://www.pythontutorial.net/python-concurrency/python-threading-lock/

illustrates a race condition problem: Output will vary 10 & 20 each time you run , depending race

In [ ]:
from threading import Thread
from time import sleep


counter = 0

# define a function that increases the value of the counter variable by a number:
def increase(by):
    global counter

    local_counter = counter
    local_counter += by

    sleep(0.1)

    counter = local_counter
    print(f'counter={counter}\n')


# create two threads. 
# first thread increases the counter by 10 
# second thread increases the counter by 20:
t1 = Thread(target=increase, args=(10,))
t2 = Thread(target=increase, args=(20,))

# start the threads
t1.start()
t2.start()


# wait for the threads to complete
t1.join()
t2.join()


print(f'The final counter is {counter}')

#### Solution using mutext lock
#### Using a threading lock to prevent the race condition

In Python, you can use the `Lock` class from the `threading` module to create a lock object:

First, create an instance the Lock class:

```python
lock = Lock()
```

By default, the lock is unlocked until a thread acquires it.

Second, acquire a lock by calling the acquire() method:

```python
lock.acquire()
```

Third, release the lock once the thread completes changing the shared variable:

```python
lock.release()
```

In [ ]:
from threading import Thread, Lock
from time import sleep

# Initialize the global counter variable
counter = 0

# Define the thread function to increase the counter by a given value using a lock
def increase(by, lock):
    global counter

    # Acquire the lock to ensure exclusive access to the shared counter
    lock.acquire()

    # Create a local copy of the counter to perform the update
    local_counter = counter
    local_counter += by

    # Simulate some time-consuming work using sleep
    sleep(0.1)

    # Update the global counter with the new value
    counter = local_counter
    print(f'counter={counter}')

    # Release the lock to allow other threads to access the shared counter
    lock.release()

# Create a Lock object to synchronize access to the shared counter
lock = Lock()

# Create two threads, each incrementing the counter by a different value
t1 = Thread(target=increase, args=(10, lock))
t2 = Thread(target=increase, args=(20, lock))

# Start the threads
t1.start()
t2.start()

# Wait for the threads to complete their execution
t1.join()
t2.join()

# Print the final value of the counter
print(f'The final counter is {counter}')


It’s easier to use the lock object with the with statement to acquire and release the lock within a block of code:

In [ ]:
from threading import Thread, Lock
from time import sleep

counter = 0

def increase(by, lock):
    global counter

    # Use the 'with' statement to acquire and release the lock automatically
    with lock:
        # Create a local copy of the counter to perform the update
        local_counter = counter
        local_counter += by

        # Simulate some time-consuming work using sleep
        sleep(0.1)

        # Update the global counter with the new value
        counter = local_counter
        print(f'counter={counter}')

lock = Lock()

# create threads
t1 = Thread(target=increase, args=(10, lock))
t2 = Thread(target=increase, args=(20, lock))

# start the threads
t1.start()
t2.start()

# wait for the threads to complete
t1.join()
t2.join()

# Print the final value of the counter
print(f'The final counter is {counter}')


### Alternative (same but reworked code)

Alternative code follows a more structured and object-oriented approach by encapsulating the shared counter and lock within a class. This approach is more modular and reusable, allowing you to easily create multiple instances of the Counter class with their own separate counters and locks, each capable of independent synchronization

In [ ]:
from threading import Thread, Lock
from time import sleep

# Define a class to encapsulate the shared counter and lock
class Counter:
    def __init__(self):
        # Initialize the counter value to 0
        self.value = 0
        # Create a lock object to synchronize access to the counter
        self.lock = Lock()

    # Method to increase the counter by a specified value
    def increase(self, by):
        # Acquire the lock to ensure exclusive access to the shared counter
        with self.lock:
            # Create a local variable to perform the update operation
            current_value = self.value
            # Increment the local variable by the specified value
            current_value += by

            # Simulate some time-consuming work using sleep
            sleep(0.1)

            # Update the shared counter with the new value
            self.value = current_value
            # Print the updated value of the counter
            print(f'counter={self.value}')

# Main function
def main():
    # Create an instance of the Counter class to manage the shared counter
    counter = Counter()

    # Create two threads with different increment values
    t1 = Thread(target=counter.increase, args=(10, ))
    t2 = Thread(target=counter.increase, args=(20, ))

    # Start the threads
    t1.start()
    t2.start()

    # Wait for the threads to complete their work
    t1.join()
    t2.join()

    # Print the final value of the counter after both threads have finished
    print(f'The final counter is {counter.value}')

# Check if the script is run directly (not imported as a module)
if __name__ == '__main__':
    # Call the main function to start the threads and perform the synchronization
    main()


                                                Another Example

#### Synchronization Errors
We will create two threads and give each of them a pointer towards a variable in the main containing an unsigned integer, count. Each thread will iterate a certain number of times (defined in the TIMES_TO_COUNT macro) and increment the count at each iteration. Since there are two threads, we will of course expect the final count to be exactly twice TIMES_TO_COUNT.

##### Solution using mutext lock

In [ ]:
import threading
import time

# Each thread will count TIMES_TO_COUNT times
TIMES_TO_COUNT = 21000

class Counter:
    def __init__(self):
        self.count = 0
        self.lock = threading.Lock()

    def increment(self):
        with self.lock:
            self.count += 1

def thread_routine(counter):
    # Each thread starts here
    tid = threading.current_thread().ident
    # Print the count before this thread starts iterating.
    # In order to read the value of count, we lock the mutex:
    with counter.lock:
        print(f"Thread [{tid}]: Count at thread start = {counter.count}")
    for i in range(TIMES_TO_COUNT):
        # Iterate TIMES_TO_COUNT times
        # Increment the counter at each iteration
        # Lock the mutex for the duration of the incrementation
        counter.increment()
    # Print the final count when this thread finishes its own count,
    # without forgetting to lock the mutex:
    with counter.lock:
        print(f"Thread [{tid}]: Final count = {counter.count}")

def main():
    # Structure containing the threads' total count:
    counter = Counter()

    # Since each thread counts TIMES_TO_COUNT times and that
    # we have 2 threads, we expect the final count to be
    # 2 * TIMES_TO_COUNT:
    print(f"Main: Expected count is {2 * TIMES_TO_COUNT}")
    # Thread creation:
    t1 = threading.Thread(target=thread_routine, args=(counter,))
    print(f"Main: Created first thread [{t1.ident}]")
    t2 = threading.Thread(target=thread_routine, args=(counter,))
    print(f"Main: Created second thread [{t2.ident}]")
    # Thread starting:
    t1.start()
    t2.start()
    # Thread joining:
    t1.join()
    print(f"Main: Joined first thread [{t1.ident}]")
    t2.join()
    print(f"Main: Joined second thread [{t2.ident}]")
    # Final count evaluation:
    # (Here we can read the count without worrying about
    # the lock because all threads have been joined and
    # there can be no data race between threads)
    if counter.count != (2 * TIMES_TO_COUNT):
        print(f"Main: ERROR! Total count is {counter.count}")
    else:
        print(f"Main: OK. Total count is {counter.count}")

if __name__ == "__main__":
    main()
